In [18]:
import numpy
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import BatchNormalization
from keras.constraints import maxnorm
from keras.optimizers import SGD
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
import numpy as np
import time

In [3]:
tf.__version__

'1.14.0'

In [6]:
os.getcwd()

'C:\\Users\\Dhirender.Jit'

In [7]:
def get_images(directory):
    Images = []
    Labels = []
    for dir_name in os.listdir(directory): 
        for image_file in os.listdir(directory+dir_name):
            image = cv2.imread(directory+dir_name+r'/'+image_file)
            if image is not None:
                image = cv2.resize(image,(300,300),)
                Images.append(image)
                Labels.append(dir_name)
    return Images, Labels

In [10]:
Images, Labels = get_images('C:/Users/Dhirender.Jit/plant-seedlings-classification/train/')

In [20]:
labels = []
mapping = { 'Sugar beet': 0, 'Fat Hen': 1, 'Scentless Mayweed' : 2, 'Charlock' : 3,
           'Small-flowered Cranesbill': 4, 'Maize': 5, 'Shepherds Purse' :6, 'Common wheat': 7,
           'Common Chickweed': 8, 'Cleavers': 9, 'Loose Silky-bent' : 10, 'Black-grass': 11 }
for label in Labels:
    labels.append(mapping[label])
del Labels 

In [21]:
Images[0].shape

(300, 300, 3)

In [22]:
Images = np.reshape(Images,(-1,300,300,3))
Labels = np.array(labels)

In [23]:
print("Shape of training data: ", Images.shape)
print("Shape of labels data: ", Labels.shape)

Shape of training data:  (4750, 300, 300, 3)
Shape of labels data:  (4750,)


In [24]:
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(Images, Labels, test_size=.2, random_state=42, stratify = Labels)

In [25]:
from keras.utils import np_utils
y_train = np_utils.to_categorical(y_train,num_classes=12)
y_val = np_utils.to_categorical(y_val,num_classes=12)

In [26]:
print(x_train.shape)
print(y_train.shape)
print(x_val.shape)
print(y_val.shape)

(3800, 300, 300, 3)
(3800, 12)
(950, 300, 300, 3)
(950, 12)


In [27]:
# normalize inputs from 0-255 to 0.0-1.0
X_train = x_train.astype('float32')
X_val = x_val.astype('float32')
X_train = x_train / 255.0
X_val = x_val / 255.0

In [28]:
# Create the model
model = Sequential()
model.add(Conv2D(120, (3, 3), input_shape=(300, 300, 3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(100, (3, 3), activation='relu', padding='same'))
model.add(BatchNormalization()) 
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(80, (3, 3), activation='relu', padding='same'))
model.add(BatchNormalization()) 
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(60, (3, 3), activation='relu', padding='same'))
model.add(BatchNormalization()) 
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(40, (3, 3), activation='relu', padding='same'))
model.add(BatchNormalization()) 
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(20, (3, 3), activation='relu', padding='same'))
model.add(BatchNormalization()) 
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(12, activation='softmax'))

W0817 10:26:07.552195 12928 deprecation_wrapper.py:119] From C:\Users\Dhirender.Jit\AppData\Roaming\Python\Python37\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0817 10:26:07.572196 12928 deprecation_wrapper.py:119] From C:\Users\Dhirender.Jit\AppData\Roaming\Python\Python37\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0817 10:26:07.580248 12928 deprecation_wrapper.py:119] From C:\Users\Dhirender.Jit\AppData\Roaming\Python\Python37\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0817 10:26:07.600234 12928 deprecation_wrapper.py:119] From C:\Users\Dhirender.Jit\AppData\Roaming\Python\Python37\site-packages\keras\backend\tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf

In [29]:
epochs = 40

In [30]:
# Compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

W0817 10:28:59.640022 12928 deprecation_wrapper.py:119] From C:\Users\Dhirender.Jit\AppData\Roaming\Python\Python37\site-packages\keras\optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 300, 300, 120)     3360      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 150, 150, 120)     0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 150, 150, 100)     108100    
_________________________________________________________________
batch_normalization_1 (Batch (None, 150, 150, 100)     400       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 75, 75, 100)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 75, 75, 80)        72080     
_________________________________________________________________
batch_normalization_2 (Batch (None, 75, 75, 80)        320       
__________

In [ ]:
# Fit the model
start = time.clock() 
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=epochs, batch_size=32)
#model.fit(X_train, y_train, validation_split=0.3, epochs=epochs, batch_size=32)
end = time.clock()

print("Train Time: {} ".format(end-start)) 

C:\Users\Dhirender.Jit\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  
W0817 10:29:58.951084 12928 deprecation.py:323] From C:\Users\Dhirender.Jit\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 3800 samples, validate on 950 samples
Epoch 1/40
3800/3800 [==============================] - 2197s 578ms/step - loss: 1.8120 - acc: 0.4232 - val_loss: 1.6424 - val_acc: 0.4853
Epoch 2/40
 992/3800 [======>.......................] - ETA: 20:27 - loss: 1.0564 - acc: 0.6633

In [ ]:
loss_and_metrics = model.evaluate(X_test, y_test)
print(loss_and_metrics)